# Load Libraries

In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
import seaborn as sns
import pandas as pd
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
%matplotlib inline
import numpy as np
import scipy as sp
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
import ast
import pickle
from sklearn import tree
import pydot

/home/dimitris/anaconda3/envs/ztdl/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Load Datasets

In [2]:
# Loading the first dataset
X = pd.read_csv('normalized_generic.csv',sep='\t')
X = X.drop('Unnamed: 0',axis=1)


# Merging the two datasets
a=X.loc[X['rumor'] == 0.0]
b=X.loc[X['rumor'] == 1.0]
a=a[0:len(b)]
X=pd.concat([a,b])
X = shuffle(X)
X= X.reset_index(drop=True)

# Defining the last column of the dataframe as the output column of the model
y=X['rumor']

X = X.iloc[:, :-1]

In [ ]:
metacontent_features= X.iloc[:,:8]
metacontent_features['rumor']=X.rumor
metacontent_features.to_csv('metacontent_features.csv', sep='\t')
user_features= X.iloc[:,8:32]
user_features['rumor']=X.rumor
user_features.to_csv('user_features.csv', sep='\t')
linguistic_features= X.iloc[:,32:]
linguistic_features.to_csv('linguistic_features.csv', sep='\t')

In [17]:
a=X.loc[X['rumor'] == 0.0]
b=X.loc[X['rumor'] == 1.0]
a=a[0:len(b)]
X=pd.concat([a,b])
X = shuffle(X)
X= X.reset_index(drop=True)

# Rumor Detector

## Desicion Tree Experimentation

In [5]:
# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [ 2, 4, 6, 8, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8]

# Create the random grid
random_grid = {'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [ ]:
dt = DecisionTreeClassifier(random_state = 42)
 
# Define multiple metrics to be stored during the GridSearch
scoring = {'accuracy':'accuracy','precision':'precision', 'f1':'f1', 'recall':'recall'}

# Random search of parameters, using 5 fold cross validation
dt_random_truth = GridSearchCV(estimator=dt, param_grid=random_grid, cv=5, scoring= scoring, 
                               n_jobs=8, refit='recall')

# Fit the random search model
dt_random_truth.fit(X,y)

### Saving the results

In [13]:
output_truth_dt = open('dt_rumour.pkl', 'wb')

pickle.dump(dt_random_truth.cv_results_, output_truth_dt)

output_truth_dt.close()

## SVM Experimentation

In [4]:
# Exploring the values of C in logarithmic progression
Cs = [0.001, 0.01, 0.1, 1, 10]

# Exploring the values of Gamma in logarithmic progression
gammas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# Exploring the most suitable kernel
kernel = ['rbf', 'linear']

param_grid = {'C': Cs, 'gamma' : gammas, 'kernel': kernel}

In [ ]:
# Defining the model
svm = SVC(random_state = 42)

# Define multiple performance metrics to be stored in the GridSearch
scoring = {'accuracy':'accuracy','precision':'precision', 'f1':'f1', 'recall':'recall'}

# Random search of parameters, using 5 fold cross validation 
grid_search_truth_svm = GridSearchCV(estimator=svm, param_grid=random_grid, cv=5, scoring= scoring, 
                                     pre_dispatch=8, refit='recall')

# Fit the model
grid_search_truth_svm.fit(X,y)

### Saving the results

In [ ]:
output_truth_svm = open('svm_rumour.pkl', 'wb')

pickle.dump(grid_search_truth_svm.cv_results_, output_truth_svm)

output_truth_svm.close()

# Humanitarian Relevancy Estimator

## Loading Datasets

In [2]:
X1 = pd.read_csv('generic_text_all.csv',sep='\t', header=None, index_col=False, lineterminator='\n')
X1 = X1.drop(0,axis=1)
X1 = X1.drop(0)
X1 = X1.reset_index(drop=True)
X1['crisis'] = 0.0 

X2 = pd.read_csv('humanitarian_text.csv',sep='\t', header=None, index_col=False, lineterminator='\n')
X2 = X2.drop(0,axis=1)
X2 = X2.reset_index(drop=True)
X2['crisis'] = 1.0


X=pd.concat([X1, X2])
X= X.reset_index(drop=True)
y = X['crisis']

## Loading Unigram

In [21]:
sp.sparse.save_npz('unigram.npz', messages_tfidf)

## Loading Bigram

In [22]:
sparse_matrix = sp.sparse.load_npz('bigram.npz')

## Multinomial Naive Bayes Experimentation

In [23]:
a = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

fit = [True, False]

parameters = {'alpha': a, 'fit_prior': fit}

In [24]:
mnb = MultinomialNB()
scoring = {'accuracy':'accuracy','precision':'precision', 'f1':'f1', 'recall':'recall'}
grid_search2 = GridSearchCV(mnb, parameters, cv=5, scoring=scoring, n_jobs= 8, refit='recall',return_train_score=True)
grid_search2.fit(sparse_matrix,y)

/home/dimitris/anaconda3/envs/ztdl/lib/python3.5/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/home/dimitris/anaconda3/envs/ztdl/lib/python3.5/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/home/dimitris/anaconda3/envs/ztdl/lib/python3.5/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/home/dimitris/anaconda3/envs/ztdl/lib/python3.5/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/home/dimitris/anaconda3/envs/ztdl/lib/python3.5/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric e

GridSearchCV(cv=5, error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid=True, n_jobs=8,
       param_grid={'alpha': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'fit_prior': [True, False]},
       pre_dispatch='2*n_jobs', refit='recall', return_train_score=True,
       scoring={'f1': 'f1', 'precision': 'precision', 'recall': 'recall', 'accuracy': 'accuracy'},
       verbose=0)

### Save the results

In [25]:
output1 = open('mnb_unigram.pkl', 'wb')

pickle.dump(grid_search2.cv_results_, output1)

output1.close()

## Random Forest Experimentation

## Decision Tree Experimentation

In [4]:
# Number of features to consider at every split
#max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [10,100,1000,10000]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 4, 6, 8, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8]

# Create the random grid
random_grid = {'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

In [ ]:
dt = DecisionTreeClassifier(random_state = 42)
# Random search of parameters, using 5 fold cross validation, 
scoring = {'accuracy':'accuracy','precision':'precision', 'f1':'f1', 'recall':'recall'}
dt_random = GridSearchCV(estimator=dt, param_grid=random_grid, cv=5, scoring= scoring, 
                                     n_jobs=4, refit='accuracy')

# Fit the random search model
dt_random.fit(sparse_matrix,y)

In [ ]:
dt_random.cv_results_

### Save the results

In [8]:
output3 = open('dt_unigram_only.pkl', 'wb')

pickle.dump(dt_random.cv_results_, output3)

output3.close()